In [1]:
import cv2
from ultralytics import YOLO  # Ensure YOLO is imported correctly
import numpy as np

In [3]:
import cuda


ModuleNotFoundError: No module named 'cuda'

In [2]:
try:
    import cuda
    print("CUDA-Python is installed.")
except ImportError:
    print("CUDA-Python is not installed.")


CUDA-Python is not installed.


In [20]:
class State:
    WAIT_FOR_PICKUP = 1
    WAIT_FOR_DROP = 2

def draw_sides(frame, region, sides, color):
    x_start, y_start, x_end, y_end = region[0][0], region[0][1], region[1][0], region[1][1]
    thickness = 2

    if 'left' in sides:
        cv2.line(frame, (x_start, y_start), (x_start, y_end), color, thickness)
    if 'right' in sides:
        cv2.line(frame, (x_end, y_start), (x_end, y_end), color, thickness)
    if 'top' in sides:
        cv2.line(frame, (x_start, y_start), (x_end, y_start), color, thickness)
    if 'bottom' in sides:
        cv2.line(frame, (x_start, y_end), (x_end, y_end), color, thickness)

def point_line_distance(point, line_start, line_end):
    """Calculate the minimum distance from a point to a line segment."""
    # Line vector
    line_vec = np.array(line_end) - np.array(line_start)
    # Point vector
    point_vec = np.array(point) - np.array(line_start)
    # Line length squared
    line_len2 = line_vec.dot(line_vec)
    # Project point onto the line using dot product
    projection = point_vec.dot(line_vec) / line_len2
    if projection < 0:
        projection = 0
    elif projection > 1:
        projection = 1
    # Find the closest point on the line segment
    closest_point = np.array(line_start) + projection * line_vec
    # Return the distance from the point to the closest point on the line
    return np.linalg.norm(closest_point - np.array(point))

def is_entering_from_side(box, region, sides, threshold=50):
    """Check if the center of a bounding box is within a threshold distance of a specified side of a region."""
    x_center, y_center = (box[0] + box[2]) / 2, (box[1] + box[3]) / 2
    
    side_centers = {
        'left': ((region[0][0], region[0][1]), (region[0][0], region[1][1])),
        'right': ((region[1][0], region[0][1]), (region[1][0], region[1][1])),
        'top': ((region[0][0], region[0][1]), (region[1][0], region[0][1])),
        'bottom': ((region[0][0], region[1][1]), (region[1][0], region[1][1]))
    }
    
    for side in sides:
        line_start, line_end = side_centers[side]
        if point_line_distance((x_center, y_center), line_start, line_end) < threshold:
            return True
    return False

def process_video(video_url, pickup_coords, drop_coords, pickup_sides, drop_sides, output_path):
    model = YOLO('handDetection.pt')
    cap = cv2.VideoCapture(video_url)
    if not cap.isOpened():
        raise ValueError(f"Couldn't open video stream from URL: {video_url}")

    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            results = model(frame, conf=0.05, iou=0.2)

            for box_data in results[0].boxes.data.cpu().numpy():
                box = list(map(int, box_data[:4]))
                entering_pickup = is_entering_from_side(box, pickup_coords, pickup_sides)
                entering_drop = is_entering_from_side(box, drop_coords, drop_sides)

                label_pickup = f"Entering Pickup: {entering_pickup}"
                label_drop = f"Entering Drop: {entering_drop}"
                cv2.putText(frame, label_pickup, (box[0], box[1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
                cv2.putText(frame, label_drop, (box[0], box[1]-25), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
                cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2)

            # Draw pickup and drop rectangles in green
            cv2.rectangle(frame, tuple(pickup_coords[0]), tuple(pickup_coords[1]), (0, 255, 0), 2)
            cv2.rectangle(frame, tuple(drop_coords[0]), tuple(drop_coords[1]), (0, 255, 0), 2)

            # Draw specified sides in red
            draw_sides(frame, pickup_coords, pickup_sides, (0, 0, 255))
            draw_sides(frame, drop_coords, drop_sides, (0, 0, 255))

            out.write(frame)

    finally:
        cap.release()
        out.release()
        cv2.destroyAllWindows()

# Example usage
video_url = 'videos/testVideo2.mp4'
output_path = 'output/test_output.mp4'
pickup_coords = [[5, 10], [120, 140]]  # Define pickup ROI coordinates
drop_coords = [[240, 60], [450, 190]]  # Define drop ROI coordinates
pickup_sides = ["right", "bottom"]      # Define pickup sides to check
drop_sides = ["left", "bottom"]                 # Define drop sides to check

process_video(video_url, pickup_coords, drop_coords, pickup_sides, drop_sides, output_path)

In [12]:
def process_video(video_url):
    model = YOLO('handDetection.pt')
    cap = cv2.VideoCapture(video_url)
    if not cap.isOpened():
        raise ValueError(f"Couldn't open video stream from URL: {video_url}")

    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            results = model(frame, conf=0.05, iou=0.2)

            # Debugging print statements
            print("Box Data:", results[0].boxes.data.cpu().numpy())

            for box_data in results[0].boxes.data.cpu().numpy():
                box = list(map(int, box_data[:4]))
                
                # More debugging print statements
                print("Box coordinates:", box)

                # [Include any additional processing and drawing here]

    finally:
        cap.release()
        cv2.destroyAllWindows()

# Example usage
video_url = 'videos/testVideo2.mp4'
# output_path = 'output/test_output.mp4'
process_video(video_url)